### Fatal Encounters
<!-- ```{python}
import pandas as pd
import plotly.express as px
from urllib.request import urlopen
import json
```

In [ ]:
sheet_id = "1dKmaV_JiWcG8XBoRgP8b4e9Eopkpgt7FL7nyspvzAsE"
sheet_name = "sample_1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = pd.read_csv(url)
ojs_define(fe_data = df)

df['Age'].fillna('Not in Fatal Encounters dataset')
df_ca_only = df.loc[df['State'] == 'CA']

### California Only

```{ojs}
ca_only = transpose(fe_data).filter(function(p) {
  return p.State === 'CA';
})
```

```{ojs}
viewof fe_ca_search_all_cols = Inputs.search(ca_only, 
{placeholder: "Enter a name, zip code, county, city, or police department name", 
width: 900,
label: "Search FE Data (CA only)"
})
```

```{ojs}
viewof fe_rows_ca_all_cols = Inputs.table(fe_ca_search_all_cols, 
{
  columns: [
    'Unique ID',
    'Name',
    'Age',
    'Gender',
    'Race',
    'Race with imputations',
    'Imputation probability',
    'URL of image (PLS NO HOTLINKS)',
    ' Date of injury resulting in death (month/day/year)',
    'Location of injury (address)',
    'Location of death (city)',
    'State',
    'Location of death (zip code)',
    'Location of death (county)',
    'Full Address',
    'Latitude',
    'Longitude',
    'Agency or agencies involved',
    'Highest level of force',
    'UID Temporary',
    'Name Temporary',
    'Armed/Unarmed',
    'Alleged weapon',
    'Aggressive physical movement',
    'Fleeing/Not fleeing',
    'Description Temp',
    'URL Temp',
    'Brief description',
    'Intended use of force (Developing)',
    'Supporting document link',
  ],
  format: {
    'Unique ID': x => x.toFixed(0),
    'Location of death (zip code)': x => x.toFixed(0),
    'Supporting document link': x => htl.html`<a href=${x} target=_blank>${x}</a>`,
    'URL of image (PLS NO HOTLINKS)': x => htl.html`<a href=${x} target=_blank>${x}</a>`
  },
} 
)
```

```{ojs}
abbreviated_states_obj = ({
  AL: 'Alabama',
  AK: 'Alaska' ,
  AS: 'American Samoa',
  AZ: 'Arizona',
  AR: 'Arkansas',
  CA: 'California',
  CO: 'Colorado',
  CT: 'Connecticut',
  DE: 'Delaware',
  DC: 'District of Columbia',
  FL: 'Florida',
  GA: 'Georgia',
  GU: 'Guam',
  HI: 'Hawaii',
  ID: 'Idaho',
  IL: 'Illinois',
  IN: 'Indiana',
  IA: 'Iowa',
  KS: 'Kansas',
  KY: 'Kentucky',
  LA: 'Louisiana',
  ME: 'Maine',
  MD: 'Maryland',
  MA: 'Massachusetts',
  MI: 'Michigan',
  MN: 'Minnesota',
  MS: 'Mississippi',
  MO: 'Missouri',
  MT: 'Montana',
  NE: 'Nebraska',
  NV: 'Nevada',
  NH: 'New Hampshire',
  NJ: 'New Jersey',
  NM: 'New Mexico',
  NY: 'New York',
  NC: 'North Carolina',
  ND: 'North Dakota',
  OH: 'Ohio',
  OK: 'Oklahoma',
  OR: 'Oregon',
  PA: 'Pennsylvania',
  PR: 'Puerto Rico',
  RI: 'Rhode Island',
  SC: 'South Carolina',
  SD: 'South Dakota',
  TN: 'Tennessee',
  TX: 'Texas',
  UT: 'Utah',
  VT: 'Vermont',
  VI: 'Virgin Islands',
  VA: 'Virginia',
  WA: 'Washington',
  WV: 'West Virginia',
  WI: 'Wisconsin',
  WY: 'Wyoming',
})
tidy_w_race = transpose(fe_data).map(d => ({date: new Date(d[' Date of injury resulting in death (month/day/year)']).getFullYear(), count: 1, raceImputed: d['Race with imputations'], race: d.Race, state: abbreviated_states_obj[d.State]}))
```


## Preliminary FE Data Visualizations
### Entire U.S.
The plot below visualizes the number of fatal encounters per year for all of the United States.


```{ojs}
viewof race_imputed = Inputs.radio(new Map([["Race with imputations", 'raceImputed'], ["Race with no imputations", 'race']]), {value: 'raceImputed', label: "Race imputed?"})
```

```{ojs}
Plot.plot({
  width: 928,
  height: 500,
  x: {tickFormat: ""},
  y: {tickSpacing: 50},
  color: {legend: true},
  marks: [
  Plot.barY(tidy_w_race, {x: 'date', y: 'count', fill: race_imputed, sort: race_imputed}),
  ]
})
```

The plot below visualizes the number of fatal encounters per year for all of the United States separated by state.

```{ojs}
tidy_no_race = tidy_w_race.map(d => ({date: d.date, count: d.count, state: d.state}))
```

```{ojs}
viewof fe_search_by_state = Inputs.checkbox(d3.group(tidy_no_race, p => p.state), {key: ['California', 'Georgia'], sort: "ascending", label: html`<b>Select 1 or more states</b>`})
```

```{ojs}
Plot.plot({
  width: 928,
  height: 500,
  x: {tickFormat: ""},
  y: {tickSpacing: 50},
  color: {legend: true},
  marks: [
  Plot.barY(fe_search_by_state.flat(), {x: 'date', y: 'count', fill: 'state', sort: 'state'}),
  ]
})
```


### California Only
The plot below visualizes the number of fatal encounters per year for **California only**. 

**Note**: the "race" column is derived from Fatal Encounter's "Race with imputations" column

```{ojs}
ca_only_tidy = ca_only.map(p => ({name: p.Name, date: new Date(p[' Date of injury resulting in death (month/day/year)']).getFullYear(), count: 1, race: p['Race with imputations'], lea: p['Agency or agencies involved']}))
```

```{ojs}
viewof fe_ca_search = Inputs.search(ca_only_tidy, 
{placeholder: "Enter a name, year, race, or police department name", 
width: 900,
label: "Search Fatal Encounter Dataset (CA only)"
})
```

```{ojs}
viewof fe_rows_ca = Inputs.table(fe_ca_search, 
{
  columns: [
    'name',
    'date',
    'race',
    'lea',
  ],
  format: {
    date : x => x.toFixed(0),
    // 'Location of death (zip code)': x => x.toFixed(0),
    // 'Supporting document link': x => htl.html`<a href=${x} target=_blank>${x}</a>`,
    // 'URL of image (PLS NO HOTLINKS)': x => htl.html`<a href=${x} target=_blank>${x}</a>`
  },
  header: {
    name: 'Name',
    date: 'Date of death',
    race: 'Race with imputations',
    lea: 'Law Enforcement Agency/Agencies involved'
  }
} 
)
```

Plot.plot({
  width: 1000,
  height: 600,
  x: {tickFormat: ""},
  y: {tickSpacing: 50},
  color: {legend: true},
  marks: [
  Plot.barY(fe_ca_search, {x: 'date', y: 'count', fill: 'race', sort: 'race'}),
  ]
})
``` -->